# FYP15002 Chinese character and word analysis in daily essays - Natural Langugage Processing

# Preamble

### About Author
- Milton Leung
- 3035053381
- mingtak@connect.hku.hk

### About presentation slides
- HTML code produced by ipython notebook
- Powered by reveal.js

### About presentation
- Background: Applications, NLP Pipeline
- Chinese NLP: Segmentation, algorithms and demonstration
- Case Study: Training models on Chinese Wikipedia


### About working environment
- Unix-like environment
- Minimum 2GB RAMs
- Python 2.7

### Aouut packages and tools
- External packages: gensim, jieba; pip install gensim, jieba
- Working tools: IPython Notebook and Jupyter: An interactive computational environment

Image source: <a href=http://ipython.org/_static/sloangrant/9_home_fperez_prof_grants_1207-sloan-ipython_proposal_fig_ipython-notebook-specgram.png>http://ipython.org/_static/sloangrant/9_home_fperez_prof_grants_1207-sloan-ipython_proposal_fig_ipython-notebook-specgram.png</a>
<img src=http://ipython.org/_static/sloangrant/9_home_fperez_prof_grants_1207-sloan-ipython_proposal_fig_ipython-notebook-specgram.png width=50% height=50%>

In [1]:
# -*- coding: utf-8 -*-
import codecs
import sys
import unicodedata
import pickle
import jieba
import jieba.posseg as pseg
from langconv import *
from gensim import corpora, models, similarities
from gensim.models import Word2Vec

# Background

### Project Pipeline
<img src=img/ProjectFlow.png>

### Why NLP?
- Support searching and retrieval of documents
- Extract key sentences from documents
- Identify similar documents
- Identify major topics of in a group of documents
- Detect sentiment in user comments
- ...
- NLP is fun!

# Chinese Natural Language Processing

### Chinese NLP 
- Sentence = "你好世界"
- How to segment a sentence into words?
- (你好世界), (你好 / 世界), (你 / 好世界),  (你好世 / 界), (你 / 好 / 世界)  ......
<img src=http://thumbs.dreamstime.com/x/你好世界-11059717.jpg>

### Word Segmentation Algorithm
- Based on a prefix dictionary structure, we achieve efficient word graph scanning. 
<img src=img/Picture1.png  width="50%" height="50%"> 
- Build a directed acyclic graph (DAG) for all possible word combinations.
- Use dynamic programming to find the most probable combination based on the word frequency.
<img src=img/helloWorld.png  width="50%" height="50%"> 

- For unknown words, a HMM-based model is used with the Viterbi algorithm.
<img src=https://upload.wikimedia.org/wikipedia/commons/7/73/Viterbi_animated_demo.gif>

# Demo: Word Segmentation by Jieba

### Jieba (結巴) - Chinese Word Segmentation Module
- Support Simpified and Traditional Chinese Segmentations
- Allow to use user defined Dictionaries
- MIT License
- GitHub: <a href=https://github.com/fxsjy/jieba>https://github.com/fxsjy/jieba</a>


In [3]:
helloWorld = "你好世界"
print " / ".join(jieba.cut(helloWorld))

你好 / 世界


### Part-of-Speech Tagging 
<a href=https://gist.github.com/luw2007/6016931#ictclas-汉语词性标注集>https://gist.github.com/luw2007/6016931#ictclas-汉语词性标注集</a>
- l	习用语	习用语尚未成为成语，有点“临时性”，取“临”的声母。
- n	名词	取英语名词noun的第1个字母。


In [4]:
words = pseg.cut(helloWorld)
for word, flag in words:
    print word, flag

你好 l
世界 n


In [6]:
# Chinese dictionary from Dr. Vincent Lau
!echo "`head -5 dict`\n......\n`tail -5 dict`"

一  般 直 些 時 定 月 點 一 旦 切 樣 再 起 致 度 路 向 方面 處 帶 面 併 塊 律 連串 邊 舉 生 口氣 味 貫 道 同 陣子 則 連 旁 流 手 體 共 般而言 來 如 早 經 下子 心 角 大早 概 窩蜂 席之地 干 時之間 觸即發 字 發不可收拾 枝獨秀 會兒 會 落千丈 面倒 心一意 帆風順 模一樣 髮千鈞 舉兩得 蹶不振 籌莫展 覽無遺
乙  烯 炔 二醇 卯 酸
丁  二烯 烷 香 憂
七  星 彩 色 嘴八舌 折八扣 情 零八落 情六慾 竅 言絕句 言律詩 彩繽紛 十二烈士 手八腳 律 拼八湊 里香
乃  至 父 弟
......
攢  錢
鱈  魚
鰲  頭
鷸  蚌相爭 蚌相爭漁翁得利
顴  骨


In [7]:
!echo "`head -5 dict.txt`\n......\n`tail -5 dict.txt`"

一般
一直
一些
一時
一定
......
鱈魚
鰲頭
鷸蚌相爭
鷸蚌相爭漁翁得利
顴骨


In [3]:
sentence = """數學（Mathematics）是利用符號語言研究數量、結構、變化以及空間等概念的一門學科，
從某種角度看屬於形式科學的一種。數學透過抽象化和邏輯推理的使用，由計數、計算、量度和對物體形狀及運動的觀察而產生。
數學家們拓展這些概念，為了公式化新的猜想以及從選定的公理及定義中建立起嚴謹推導出的定理。"""
segment1= " / ".join(jieba.cut(sentence))
print segment1

數學 / （ / Mathematics / ） / 是 / 利用 / 符號 / 語言 / 研究 / 數量 / 、 / 結構 / 、 / 變化 / 以及 / 空間 / 等 / 概念 / 的 / 一門 / 學科 / ， / 
 / 從 / 某種 / 角度看 / 屬 / 於 / 形式 / 科學 / 的 / 一種 / 。 / 數學 / 透過 / 抽象化 / 和 / 邏輯 / 推理 / 的 / 使用 / ， / 由計數 / 、 / 計算 / 、 / 量度 / 和 / 對 / 物體 / 形狀及 / 運動 / 的 / 觀察而產生 / 。 / 
 / 數學家們 / 拓展 / 這些 / 概念 / ， / 為 / 了 / 公式化 / 新 / 的 / 猜想 / 以及 / 從 / 選定 / 的 / 公理 / 及定 / 義中 / 建立 / 起 / 嚴謹 / 推導出 / 的 / 定理 / 。


In [4]:
# Let's include our dictionaries!
jieba.load_userdict("dict.txt.big.txt")
jieba.load_userdict("dict.txt")

In [6]:
import jieba.analyse
segment2=" / ".join(jieba.cut(sentence))
print "After:\n"+ segment2
tags = jieba.analyse.extract_tags(segment2, 5)
print ",".join(tags)

After:
數學 / （ / Mathematics / ） / 是 / 利用 / 符號語言 / 研究 / 數量 / 、 / 結構 / 、 / 變化 / 以及 / 空間 / 等 / 概念 / 的 / 一門 / 學科 / ， / 
 / 從 / 某種 / 角度看 / 屬於 / 形式 / 科學 / 的 / 一種 / 。 / 數學 / 透過 / 抽象化 / 和 / 邏輯推理 / 的 / 使用 / ， / 由 / 計數 / 、 / 計算 / 、 / 量度 / 和 / 對 / 物體 / 形狀 / 及 / 運動 / 的 / 觀察 / 而 / 產生 / 。 / 
 / 數學家 / 們 / 拓展 / 這些 / 概念 / ， / 為 / 了 / 公式化 / 新 / 的 / 猜想 / 以及 / 從 / 選定 / 的 / 公理 / 及 / 定義 / 中 / 建立起 / 嚴謹 / 推導 / 出 / 的 / 定理 / 。
數學,一門,數學家,變化,計算


# Case Study:  Training Models on Chinese Wikipedia by Gensim


## Case Study:  Training Models on Chinese Wikipedia by Gensim
- Data Source: <a href=http://dumps.wikimedia.org/zhwiki/latest/>http://dumps.wikimedia.org/zhwiki/latest/</a>
- We use zhwiki-latest-abstract.xml as training dataset

### NLP Pipeline
- Data Preparation: Word extraction
- Preprocessing: Word normalization, segmentation etc. 
- Modelling: Training, validaton, analysis etc.
<img src="http://www.cis.uni-muenchen.de/~davidk/ap/res/img/nlp-pipeline.png">

# Data Cleaning and Normalization

In [11]:
# Data Source: wiki-dumped abstracts up to 4 January 2016
# Uncompressed xml file, around 1GB
!du -h zhwiki-latest-abstract.xml; 
!head -5 zhwiki-latest-abstract.xml; 
!wc -l zhwiki-latest-abstract.xml

997M	zhwiki-latest-abstract.xml
<feed>
<doc>
<title>Wikipedia：数学</title>
<url>https://zh.wikipedia.org/wiki/%E6%95%B0%E5%AD%A6</url>
<abstract>数学（Mathematics）是利用符号语言研究數量、结构、变化以及空间等概念的一門学科，从某种角度看屬於形式科學的一種。數學透過抽象化和邏輯推理的使用，由計數、計算、量度和對物體形狀及運動的觀察而產生。數學家們拓展這些概念，為了公式化新的猜想以及從選定的公理及定義中建立起嚴謹推導出的定理。Jourdain</abstract>
 14134285 zhwiki-latest-abstract.xml


### Data Cleaning and Normalization
- Extracted target tags from xml
- Filter out short abstracts
- Convert simplified Chinese to Traditional Chinese

In [12]:
# After extracting xml tags
# Over 227,000 abstracts extracted
!head -5 texts.txt

數學（Mathematics）是利用符號語言研究數量、結構、變化以及空間等概念的一門學科，從某種角度看屬於形式科學的一種。數學透過抽象化和邏輯推理的使用，由計數、計算、量度和對物體形狀及運動的觀察而產生。數學家們拓展這些概念，為了公式化新的猜想以及從選定的公理及定義中建立起嚴謹推導出的定理。Jourdain
哲學（，），源於日語對philosophy的翻譯，“哲學”一詞，不但可以是特指學術上針對思考方法的學問，也是指對普遍而基本的主題的研究，這些主題多與實在、存在、知識、價值、理性、心靈、語言、思想等有關。哲學與其他學科的不同之處在於其批判的方式、通常是系統化的方法，並且以理性論證為基礎。
文學是指以語言文字為工具形象化地反映藝術，包括戲劇、詩歌、小說、散文等，是文化的重要表現形式，以不同的形式（稱作體裁）表現內心情感和再現一定時期和一定地域的社會生活、文化。
歷史，或簡稱史，指人類社會過去的事件和行動，以及對這些事件行為有系統的記錄、詮釋和研究。歷史可提供今人理解過去，作為未來行事的參考依據，與倫理、哲學和藝術同屬人類精神文明的重要成果。歷史的第二個含義，即對過去事件的記錄和研究，又稱為“歷史學”，或簡稱“史學”。隸屬於歷史學或與其密切相關的學科有年代學、編纂學、家譜學、古文字學、計量歷史學、考古學、社會學和新聞學等，參見歷史學。習罡華，《歷史是什麼？——一項純形而上學的思考》詞源 ==
族群（），是指一群人，他們認為彼此共享了相同的祖先、血緣、外貌、歷史、文化、習俗、語言、地域、宗教、生活習慣與國家體驗等，因此形成一個共同的群體。為區分我族及「他者」的分類方式之一。族群含義在20世紀後有轉變，從原來以少數民族或少數族裔的意思，到後來以文化特徵區分，而最新的看法則認為族群是社會過程後的產生的結果。因此，族族可能因歷史及時空環境，基於歷史、文化、語言、地域、宗教、血緣祖先認同、行為、生物/外貌特征而形成「一群」與其它有所區別的群體。p.456 &quot;The ideas of ethnicity and ethnic group have a long history, often related to &quot;otherness&quot;. In the 20th century and beyond, the idea of what constitu

In [14]:
# After segmentation
# Over 6 million Chinese word segemnts 
# Around 49 million Chingese characters
!wc docs.txt
!head docs.txt

  227364 6932865 48963468 docs.txt
數學 利用 符號語言 研究 數量 結構 變化 以及 空間 概念 一門 學科 某種 角度看 屬於 形式 科學 一種 數學 透過 抽象化 邏輯推理 使用 計數 計算 量度 物體 形狀 運動 觀察 產生 數學家 拓展 這些 概念 公式化 猜想 以及 選定 公理 定義 建立起 嚴謹 推導 定理
哲學 源於 日語 翻譯 哲學 一詞 不但 可以 特指 學術 針對 思考 方法 學問 普遍 基本 主題 研究 這些 主題 實在 存在 知識 價值 理性 心靈 語言 思想 有關 哲學 與其 學科 不同之處 在於 批判 方式 通常 系統化 方法 並且 理性 論證 基礎
文學 指以 語言文字 工具 形象化 反映 藝術 包括 戲劇 詩歌 小說 散文 文化 重要 表現形式 不同 形式 稱作 體裁 表現 內心 情感 再現 一定 時期 一定 地域 社會 生活 文化
歷史 簡稱 人類 社會 過去 事件 行動 以及 這些 事件 行為 系統 記錄 詮釋 研究 歷史 提供 今人 理解 過去 作為 未來 行事 參考 依據 倫理 哲學 藝術 同屬 人類 精神文明 重要 成果 歷史 第二個 含義 過去 事件 記錄 研究 又稱 歷史學 簡稱 史學 隸屬於 歷史學 與其 密切相關 學科 年代學 編纂 家譜 古文字學 計量 歷史學 考古學 社會學 新聞學 參見 歷史學 習罡華 歷史 什麼 一項 形而上學 思考 詞源
族群 一群 他們 認為 彼此 共享 相同 祖先 血緣 外貌 歷史 文化 習俗 語言 地域 宗教 生活習慣 國家 體驗 因此 形成 一個 共同 群體 區分 我族 他者 分類 方式 之一 族群 含義 世紀 轉變 原來 少數民族 少數 族裔 意思 後來 文化 特徵 區分 最新 法則 認為 族群 社會 過程 產生 結果 因此 族族 可能 歷史 及時 環境 基於 歷史 文化 語言 地域 宗教 血緣 祖先 認同 行為 生物 外貌 特征 形成 一群 與其 有所區別 群體 這些 區別 我者
戲劇 演員 某個 故事 情境 對話 歌唱 動作 方式 表演 出來 藝術 戲劇 四個 元素 包括 演員 故事 情境 舞臺 表演 場地 觀眾
經濟學 一門 產品 服務 生產 分配 以及 消費 進行 研究 社會科學 這一 名詞 源於 古希臘 起初 這一 領域 稱為 政治經濟

# Dictionary and Corpora

### Build dictionary and corpora from documents
- Corpora: Collection of documents
- Document: A bag of word segments
- Word segment: Also known as token, basic unit in documents, abstracts or sentences
- Dictionary: Collection of all word segments

In [8]:
# Build corpus from abstracts
# the segmented abstracts are stored in docs.txt, one per line
infile = codecs.open("docs.txt", "r", "utf-8") 
documents = [ l.strip().split(" ") for l in infile ] # documents is a list of list
dictionary = corpora.Dictionary(documents) # generate the dictionary 
dictionary.filter_extremes(no_below=5) # ignore words appear less than 5 times
corpus = [ dictionary.doc2bow(d) for d in documents ] 
# Around 70,000 unique tokens
print dictionary 
# Export as wiki dictionary
dictionary.save_as_text("wikiDict.txt")

Dictionary(73710 unique tokens: [u'', u'\u71c8\u9b5a', u'\u5f35\u4e7e\u6587', u'\u6885\u6d1b', u'\u7a0b\u9865']...)


In [16]:
# Words token from 
print " ".join(documents[0])

數學 利用 符號語言 研究 數量 結構 變化 以及 空間 概念 一門 學科 某種 角度看 屬於 形式 科學 一種 數學 透過 抽象化 邏輯推理 使用 計數 計算 量度 物體 形狀 運動 觀察 產生 數學家 拓展 這些 概念 公式化 猜想 以及 選定 公理 定義 建立起 嚴謹 推導 定理


# Frequency Count

## Frequency Count
- Find most frequenct words from abstract "數學"
- Find most common words in Chinese Wikipedia

In [17]:
# Frequency count from first abstract "數學"

corpus[0].sort(key=lambda x: x[1], reverse=True)
for id, cnt in corpus[0]:
    print "(%s, %d) " % (dictionary[id], cnt),

(概念, 2)  (以及, 2)  (數學, 2)  (這些, 1)  (運動, 1)  (定義, 1)  (透過, 1)  (觀察, 1)  (使用, 1)  (抽象化, 1)  (公式化, 1)  (物體, 1)  (學科, 1)  (形式, 1)  (建立起, 1)  (一門, 1)  (利用, 1)  (屬於, 1)  (科學, 1)  (量度, 1)  (變化, 1)  (產生, 1)  (結構, 1)  (猜想, 1)  (空間, 1)  (公理, 1)  (角度看, 1)  (一種, 1)  (選定, 1)  (研究, 1)  (計數, 1)  (某種, 1)  (計算, 1)  (嚴謹, 1)  (定理, 1)  (拓展, 1)  (邏輯推理, 1)  (推導, 1)  (數量, 1)  (形狀, 1)  (數學家, 1) 


In [18]:
# Sort by frequency count
!sort -k 3 -rn wikiDict.txt > sortedWikiDict.txt
!head sortedWikiDict.txt

28687	位於	54606
50179	一個	44078
13039	中國	37696
55647	分佈	31973
39366	一種	29991
5334	其中	21486
28668	平方公里	20865
13277	地區	18431
10507	學名	17967
29576	棲息	16356


### Analysis apart from frequency count?
- ... Fullstop?

# Term weighting

## Term weighting
- Evaluate word importance
- Frequent words != important words
- High word count of "一個" != important
- What is importance meant by computer?

### Condition of an important word 
- The word should appear often in a document
- The word should *NOT* appear often in other documents

### TF-IDF
- Term Frequency - Inverse Document Frequency 
- Method to quantify the importance given a document
- Product of two statistics (TF x IDF)
- Central tool of search engines for scoring and ranking a document's relevance
<img src=https://upload.wikimedia.org/math/e/5/a/e5a7b43197068eddf42859f3995ebf15.png>
<img src=https://upload.wikimedia.org/math/0/2/5/0257ce95c505ab568d7898faa56a4f5c.png>
<img src=https://upload.wikimedia.org/math/b/0/6/b06a060c28253c8dd2528811c447862e.png>

In [19]:
# Let's build the TF-IDF model
tfidf = models.TfidfModel(corpus) 
corpus_tfidf = tfidf[corpus]

In [20]:
## Frequency Count
print "Frequency Count: "
for id, cnt in corpus[0]:
    print "(%s, %d) " % (dictionary[id], cnt),

Frequency Count: 
(概念, 2)  (以及, 2)  (數學, 2)  (這些, 1)  (運動, 1)  (定義, 1)  (透過, 1)  (觀察, 1)  (使用, 1)  (抽象化, 1)  (公式化, 1)  (物體, 1)  (學科, 1)  (形式, 1)  (建立起, 1)  (一門, 1)  (利用, 1)  (屬於, 1)  (科學, 1)  (量度, 1)  (變化, 1)  (產生, 1)  (結構, 1)  (猜想, 1)  (空間, 1)  (公理, 1)  (角度看, 1)  (一種, 1)  (選定, 1)  (研究, 1)  (計數, 1)  (某種, 1)  (計算, 1)  (嚴謹, 1)  (定理, 1)  (拓展, 1)  (邏輯推理, 1)  (推導, 1)  (數量, 1)  (形狀, 1)  (數學家, 1) 


In [15]:
## Term weighting by TF-IDF model
## "以及" has same frequency count with "數學" but not not important in terms of weighting
print "Term weighting by TF-IDF: "
for id, score in tfidf[corpus[0]]:
    print "(%s, %.2f) " % (dictionary[id], score),

Term weighting by TF-IDF: 
(一門, 0.14)  (數學家, 0.13)  (量度, 0.18)  (變化, 0.12)  (計算, 0.12)  (利用, 0.11)  (數學, 0.23)  (使用, 0.08)  (觀察, 0.14)  (透過, 0.12)  (數量, 0.13)  (以及, 0.12)  (形式, 0.10)  (一種, 0.04)  (物體, 0.14)  (邏輯推理, 0.22)  (公式化, 0.21)  (結構, 0.11)  (學科, 0.13)  (公理, 0.18)  (研究, 0.08)  (科學, 0.11)  (定理, 0.15)  (符號語言, 0.27)  (抽象化, 0.19)  (建立起, 0.19)  (某種, 0.14)  (猜想, 0.19)  (計數, 0.19)  (形狀, 0.13)  (定義, 0.11)  (嚴謹, 0.18)  (這些, 0.10)  (拓展, 0.17)  (選定, 0.17)  (概念, 0.22)  (推導, 0.19)  (運動, 0.10)  (角度看, 0.22)  (屬於, 0.06)  (產生, 0.10)  (空間, 0.12) 


### What's next?
- What can we model after finding important words?

# Document Summarization

### Document Summarization
- Usage: Information discovery, relation discovery, document classification
- Achieve by Latent Semetic Indexing

### Latent Semetic Indexing (LSI)
- Reduce 270,000 abstracts to 200 topics
- Unsupervised learning, same as Principle Component Analysis (PCA)
- Achieve by Single Value Decomposition (SVD)

In [59]:
# Train the LSI model using the TF-IDF transformed corpus
# Be careful! It takes up an hour to compute!
lsi = models.LsiModel(tfidf[corpus], id2word=dictionary, num_topics=200)

In [62]:
corpus_lsi = lsi[corpus_tfidf]

<img src=http://www.intechopen.com/source/html/40377/media/image11_w.jpg>
<img src=http://1.bp.blogspot.com/-pgMAHiIWvuw/Tql5HIXNdRI/AAAAAAAABLI/I2zPF5cLRwQ/s1600/clust.gif>

In [24]:
#Latent Sementic Indexing (LSI)
for i,j in lsi.show_topic(23)[:5]: print i,j

冰川 -0.398759466345
冰原 -0.380382283655
島峰 -0.380199566592
群島 0.230939721847
山峰 0.209200546951


In [25]:
for i,j in lsi.show_topic(0)[:5]: print i,j

棲息 0.279671065614
可達 0.250394935748
體長 0.236198529056
公分 0.234641148863
習性 0.224247181259


### LSI for text summarization: Ambiguous
- How can we interupt those topics?

# Topic Modelling: Latent Dirichlet Allocation

Image source: <a href=http://ww4.sinaimg.cn/large/6cbb8645jw1eoqsai9lhyj20pk0don1k.jpg>http://ww4.sinaimg.cn/large/6cbb8645jw1eoqsai9lhyj20pk0don1k.jpg</a>
<img src=http://ww4.sinaimg.cn/large/6cbb8645jw1eoqsai9lhyj20pk0don1k.jpg width=70% height=70%>

# Documents Similarity

## Documents Similarity
- Given a document, how to find the most similar one from corpus?
- Achieve by buliding similarity matrix (correlation matrix) from LSI

In [63]:
# Generate an index of the LSI space
# Correlation Matrix
index = similarities.MatrixSimilarity(corpus_lsi)
# Query the index with a document transformed into the tf-idf space # This returns a list of tuples in the form of (score, doc_id)


In [65]:
# Input document index
documentIndex = 40
print " ".join(documents[documentIndex][:20])

游戲 可以 指人 一種 娛樂活動 也可以 這種 活動 過程 游戲 道具 可以 玩具 英語 體育比賽 遊戲 一種 體育運動 遊戲 演變


In [69]:
sims = index[lsi[tfidf[corpus[documentIndex]]]]
# Sort the similarity values by their scores
sims = sorted(enumerate(sims), key=lambda item: -item[1])

In [ ]:
# print sims
#[(40, 1.0), (20733, 0.95213395), (94363, 0.95173347), (199968, 0.95007229), 
# (63281, 0.94930953), (55712, 0.94783223), (225236, 0.94558185), (7930, 0.94430256), 
# (63259, 0.94354367), (12311, 0.9380967), (2379, 0.93661648), (2254, 0.93655413), ......

# Three most similiar topics 
print "Index " + str(sims[1][0]) + ": " + " ".join(documents[sims[1][0]][:20])
print "Index " + str(sims[2][0]) + ": " + " ".join(documents[sims[2][0]][:20])
print "Index " + str(sims[3][0]) + ": " + " ".join(documents[sims[3][0]][:20])                      

In [78]:
print index.num_features

200


# Word Similarity

## Word Similarity
- "數學" + "計算機科學" = ?
- "蘋果" + "微軟" = ?

### Google's word2vec
- Proposed by Tomas Mikolov in late 2013
- Further reading: <a href=http://suanfazu.com/t/word2vec-zhong-de-shu-xue-yuan-li-xiang-jie-duo-tu-wifixia-yue-du/178>word2vec 中的数学原理详解（多图，WIFI下阅读）</a>


### word2vec: Principle
- Every word consider as a vector
- Given input vectors (words), predict most *relavant* words
<img src=http://1.bp.blogspot.com/-Q7F8ulD6fC0/UgvnVCSGmXI/AAAAAAAAAbg/MCWLTYBufhs/s1600/image00.gif  width=50% height=50%>

### Google's word2vec: workflow
- Training method: "Surfcae" learning, two-layer neural networks<br>
Image source: <a href=https://iksinc.files.wordpress.com/2015/04/screen-shot-2015-04-12-at-10-58-21-pm.png>https://iksinc.files.wordpress.com/2015/04/screen-shot-2015-04-12-at-10-58-21-pm.png</a>
<img src=https://iksinc.files.wordpress.com/2015/04/screen-shot-2015-04-12-at-10-58-21-pm.png width=50% height=50%>

In [ ]:
# Generate word vectors using the Word2vec algorithm
model = Word2Vec(documents, size=200, window=5, min_count=5, workers=4)

### model.most_similar(u"計算機科學")

In [54]:
for w, score in model.most_similar(u"計算機科學"): print w, score

信息科學 0.763425171375
數理統計 0.724284648895
理論物理 0.723909497261
數學 0.719389021397
統計學 0.712564647198
材料科學 0.706905901432
工程學 0.704143762589
應用 0.702665686607
計量經濟學 0.69039785862
通信工程 0.687664985657


### model.most_similar([u"數學",u"計算機科學"])

In [55]:
for w, score in model.most_similar([u"數學",u"計算機科學"]): print w, score

拓撲學 0.780949652195
理論物理 0.774173974991
數理統計 0.76259547472
統計學 0.750313699245
信息科學 0.747869253159
同倫 0.734885334969
計量經濟學 0.732247769833
微分方程 0.73089826107
概率論 0.729114413261
微積分 0.727130234241


### model.most_similar([u"蘋果",u"微軟"])

In [56]:
for w, score in model.most_similar(positive=[u"蘋果",u"微軟"]): print w, score

蘋果公司 0.773305773735
智能手機 0.760377526283
個人電腦 0.746866583824
超薄 0.73600679636
作業系統 0.727758347988
系列產品 0.726376593113
諾基亞公司 0.725656986237
手提電腦 0.722447395325
麥金塔 0.721516311169
首款 0.720096707344


### model.most_similar([u"梁振英"])

In [57]:
for w, score in model.most_similar([u"梁振英"]): print w, score

曾蔭權 0.827244877815
董建華 0.790594279766
香港特區 0.78056293726
何厚鏵 0.77780520916
施政報告 0.777581095695
公職 0.777228713036
民主派 0.771535158157
立法會 0.764261543751
香港基本法 0.762010157108
香港特別行政區 0.761656165123


In [58]:
print model.doesnt_match([u"物理",u"化學",u"生物",u"電腦"])

電腦


# Summary

## Milestone
- Word segmentation (Trie Tree, DAG and HMM)
- Word frequency count and weighting (TF-IDF)
- Text summarization (LSI)
- Search for similiar documents (LSI + Similiarity Matrix)
- Search for similiar words (Word2Vec)

## To-dos
- Topic modelling (LDA)
- More case studies
- Search for potential applications 

# One more thing...

## 二零一六年施政報告
## 2016 Policy Address

In [1]:
!head -25 PA2016.txt

二零一六年施政報告
創新經濟改善民生促進和諧繁榮共享
目錄
(一)引言(二)經濟
金融業航運及物流業旅遊業
專業服務
創意產業知識產權業
漁農業國家「十三五」規劃加強聯繫和合作
(三)「一帶一路」集資融資平台
商貿物流平台
貿易環境專業及基礎設施服務平台促進民心相通「一帶一路」辦公室
(四)創新及科技向下游出發
發展產業支援創科初創企業數碼及「智慧城市」創科生活基金其他配套
段落
段落
(五)房屋土地與交通運輸
房屋
土地短中期土地供應
檢討土地用途發展前礦場用地賣地計劃
中長期土地供應
新發展區和新市鎮擴展計劃將軍澳第區發展岩洞、地下空間及維港以外填海大嶼山
商業及經濟用地
九龍東
啓德發展區
長遠城市規劃


In [10]:
def ngrams(input, n):
  input = input.split(' ')
  output = []
  for i in range(len(input)-n+1):
    output.append(input[i:i+n])
  return output

In [9]:
lines = codecs.open("PA2016-words.txt", "r", "utf-8").readlines()
words=[]
for l in lines: 
    words += l.strip().split(" ")
documents.append(words)
dictionary = corpora.Dictionary(documents)
corpus = [ dictionary.doc2bow(d) for d in documents ]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [53]:
content = open('PA2016-words.txt', 'rb').read()
jieba.load_userdict('PA2016-jieba.txt')
PAtags = jieba.analyse.extract_tags(content, 30)
print ",".join(PAtags)
PAtags.index(u'一帶一路')
print "%s" % PAtags[8]

發展,服務,計劃,政府,香港,長者,加強,國家,一帶一路,資助,繼續,經濟,推動,建議,規劃,項目,企業,委員會,提供,單位,地區,醫療,相關,積極,落實,檢討,專業,內地,社會,醫院
一帶一路


In [54]:
corpus[227364].sort(key=lambda x: x[1], reverse=True)
for id, score in corpus[227364][:30]: print "(%s, %d) " % (dictionary[id], score),

(政府, 247)  (香港, 163)  (發展, 154)  (服務, 107)  (計劃, 101)  (提供, 78)  (包括, 58)  (長者, 50)  (增加, 49)  (加強, 49)  (工作, 48)  (研究, 46)  (以及, 45)  (國家, 44)  (一帶一路, 44)  (繼續, 42)  (資助, 42)  (土地, 40)  (合作, 39)  (經濟, 38)  (需要, 36)  (建議, 36)  (推動, 36)  (項目, 34)  (規劃, 34)  (企業, 33)  (中心, 32)  (改善, 31)  (委員會, 31)  (教育, 31) 


In [55]:
wikiTags=[]
for id, score in tfidf[corpus[227364]][:30]: 
    print "(%s, %.2f) " % (dictionary[id], score),
    wikiTags+=dictionary[id]

(政府, 0.40)  (香港, 0.19)  (發展, 0.23)  (服務, 0.18)  (計劃, 0.19)  (提供, 0.13)  (包括, 0.07)  (長者, 0.16)  (增加, 0.11)  (加強, 0.13)  (工作, 0.08)  (研究, 0.07)  (以及, 0.05)  (國家, 0.06)  (一帶一路, 0.21)  (繼續, 0.10)  (資助, 0.11)  (土地, 0.09)  (合作, 0.08)  (經濟, 0.06)  (需要, 0.07)  (建議, 0.09)  (推動, 0.08)  (項目, 0.06)  (規劃, 0.07)  (企業, 0.06)  (中心, 0.05)  (改善, 0.08)  (委員會, 0.05)  (教育, 0.06) 


In [56]:
c3 = [filter(lambda x: x in wikiTags, sublist) for sublist in PAtags]
for word in c3 : print "%s" % word,

發展 服務 計劃 政府 香港 長者 加強 國家 一帶一路 資助 繼續 經濟 推動 建議 規劃 項目 企業 委員會 提供  地      業 地 會 


In [65]:
# New word2Vec model including Policy Address
model = Word2Vec(documents, size=200, window=5, min_count=0, workers=4)

In [71]:
for w, score in model.most_similar([u"一帶一路",u"發展"]): print w, score

長遠 0.702973484993
城鄉規劃 0.688625693321
特殊作用 0.676903486252
擬訂 0.675324440002
知識經濟 0.668613672256
第十一個五年 0.668246924877
知識型 0.664405941963
偏鄉 0.662087261677
願景 0.651054263115
集體化 0.647202193737


# Reference

### Reference
Introduction to Latent Dirichlet Allocation: http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/<br>
Dimensionality Reduction and Latent Topic Models: http://pages.cs.wisc.edu/~jerryzhu/cs769/latent.pdf <br>
Google's word2vec: <a href=https://code.google.com/p/word2vec/>https://code.google.com/p/word2vec/</a><br>
Latent Semantic Analysis and Topic Modeling: Roads to Text Meaning <a href=http://www.jaist.ac.jp/~bao/Writings/TopicModeling2.pdf>http://www.jaist.ac.jp/~bao/Writings/TopicModeling2.pdf</a><br>
Latent Semantic Indexing (LSI) An Example: <a href=http://www1.se.cuhk.edu.hk/~seem5680/lecture/LSI-Eg.pdf>http://www1.se.cuhk.edu.hk/~seem5680/lecture/LSI-Eg.pdf</a><br>
word2vec 中的数学原理详解: <a href=http://suanfazu.com/t/word2vec-zhong-de-shu-xue-yuan-li-xiang-jie-duo-tu-wifixia-yue-du/178>http://suanfazu.com/t/word2vec-zhong-de-shu-xue-yuan-li-xiang-jie-duo-tu-wifixia-yue-du/178</a><br>
中英文维基百科语料上的Word2Vec实验: <a href=http://www.52nlp.cn/中英文维基百科语料上的word2vec实验>http://www.52nlp.cn/中英文维基百科语料上的word2vec实验</a><br>
Matrix decompositions and latent semantic indexing: <a href=http://nlp.stanford.edu/IR-book/pdf/18lsi.pdf>http://nlp.stanford.edu/IR-book/pdf/18lsi.pdf</a><br>
Chinese NLP with Open Source Tools in Python: <a href=https://github.com/albertauyeung/pyconhk2015-chinese-nlp>https://github.com/albertauyeung/pyconhk2015-chinese-nlp</a><br>
JIEBA 結巴中文斷詞: <a href=https://speakerdeck.com/fukuball/jieba-jie-ba-zhong-wen-duan-ci>https://speakerdeck.com/fukuball/jieba-jie-ba-zhong-wen-duan-ci</a>

# What next?

- Applications
- Visualization
<img src=http://d3js.org/preview.png>